In [7]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
from datetime import datetime 

# Load API key from .env file
load_dotenv()
coinapi_key = os.getenv('api_key')

exchange_id = "DERIBIT"

# CoinAPI endpoint for symbols metadata
url = f"https://rest.coinapi.io/v1/options/{exchange_id}/current"

#payload = {}
headers = {
  'Accept': 'text/plain',
  'X-CoinAPI-Key': coinapi_key
}

#response = requests.request("GET", url, headers=headers, data=payload)
response = requests.request("GET", url, headers=headers)

#print(response.text)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    
    # Filter for BTC/USD call options
    filtered_calls = []
    for option in data:
        if option.get("asset_id_base") == "BTC" and option.get("asset_id_quote") == "USD":
            underlying_price = option.get("underlying_price")
            time_expiration = option.get("time_expiration")

            for strike in option.get("strikes", []):
                if "call" in strike:
                    call_data = {
                        "symbol_id": strike["call"]["symbol_id"],
                        "time_exchange": strike["call"]["time_exchange"],
                        "time_coinapi": strike["call"]["time_coinapi"],
                        "strike_price": strike["strike_price"],
                        "ask_price": strike["call"]["ask_price"],
                        "ask_size": strike["call"]["ask_size"],
                        "bid_price": strike["call"]["bid_price"],
                        "bid_size": strike["call"]["bid_size"],
                        "underlying_price": underlying_price,
                        "time_expiration": time_expiration
                    }
                    filtered_calls.append(call_data)

    # Convert to DataFrame for better visualization
    df_calls = pd.DataFrame(filtered_calls)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(df_calls)

else:
    print(f"Error: {response.status_code}, {response.text}")


                               symbol_id                 time_exchange                  time_coinapi  strike_price  ask_price  ask_size  bid_price  bid_size  underlying_price               time_expiration
0     DERIBIT_OPT_BTC_USD_250328_85000_C  2025-02-04T10:12:58.9457653Z  2025-02-04T10:12:58.9458057Z       85000.0     0.1775      24.3     0.1735      31.7        98721.6805  2025-03-28T08:00:00.0000000Z
1     DERIBIT_OPT_BTC_USD_250328_95000_C  2025-02-04T10:12:51.8410247Z  2025-02-04T10:12:51.8410520Z       95000.0     0.1105      23.0     0.1070      27.3        98721.6805  2025-03-28T08:00:00.0000000Z
2     DERIBIT_OPT_BTC_USD_250328_65000_C  2025-02-04T10:11:33.5444068Z  2025-02-04T10:11:33.5444289Z       65000.0     0.3575      30.3     0.3530      30.3        98721.6805  2025-03-28T08:00:00.0000000Z
3     DERIBIT_OPT_BTC_USD_250328_75000_C  2025-02-04T10:10:49.4841660Z  2025-02-04T10:10:49.4841859Z       75000.0     0.2630      17.4     0.2580      23.9        98721.6805  2025

# Data Preparation

### Calculate 
### Market Price using ((ask price + bid price) / 2)
### Maturity(Years) using time_expiration and time_exchange
### Risk-Free Rate is gotten from OpenBB


### Maturity Years

In [8]:
df_calls = pd.DataFrame(filtered_calls)

#convert time columns to datetime
df_calls['time_exchange'] = pd.to_datetime(df_calls['time_exchange'])
df_calls['time_expiration'] = pd.to_datetime(df_calls['time_expiration'])

#Calculate the maturity in years
df_calls['time_to_maturity'] = (df_calls['time_expiration'] - df_calls['time_exchange']).dt.total_seconds() / (365.25 * 24 * 3600)
df_calls['time_to_maturity']

0      0.142115
1      0.142116
2      0.142118
3      0.142120
4      0.142117
         ...   
371    0.007962
372    0.007961
373    0.007965
374    0.007961
375    0.007961
Name: time_to_maturity, Length: 376, dtype: float64

### Market Price

In [9]:
df_calls['market_price'] = (df_calls['ask_price'] + df_calls['bid_price']) / 2
df_calls['market_price']

0      0.17550
1      0.10875
2      0.35525
3      0.26050
4      0.55175
        ...   
371    0.00040
372    0.00800
373    0.00025
374    0.00430
375    0.00115
Name: market_price, Length: 376, dtype: float64

In [10]:
from treasury_yield.risk_free_rate import risk_free_rate

df_calls["risk_free_rate"] = risk_free_rate
df_calls["risk_free_rate"]

0      0.04543
1      0.04543
2      0.04543
3      0.04543
4      0.04543
        ...   
371    0.04543
372    0.04543
373    0.04543
374    0.04543
375    0.04543
Name: risk_free_rate, Length: 376, dtype: float64

In [11]:
# Selecting relevant columns
calibration_data = df_calls[['strike_price', 'time_to_maturity', 'market_price', 'underlying_price', 'risk_free_rate']]

print(calibration_data)

     strike_price  time_to_maturity  market_price  underlying_price  risk_free_rate
0         85000.0          0.142115       0.17550        98721.6805         0.04543
1         95000.0          0.142116       0.10875        98721.6805         0.04543
2         65000.0          0.142118       0.35525        98721.6805         0.04543
3         75000.0          0.142120       0.26050        98721.6805         0.04543
4         45000.0          0.142117       0.55175        98721.6805         0.04543
..            ...               ...           ...               ...             ...
371      112000.0          0.007962       0.00040        98721.6805         0.04543
372      102000.0          0.007961       0.00800        98721.6805         0.04543
373      114000.0          0.007965       0.00025        98721.6805         0.04543
374      104000.0          0.007961       0.00430        98721.6805         0.04543
375      108000.0          0.007961       0.00115        98721.6805         